<a href="https://colab.research.google.com/github/JstKumar/KYKFMML_M1L2.INPYNB/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [1]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [2]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [3]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [4]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [5]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [6]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [7]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [8]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [9]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [10]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
When working with machine learning models, the percentage of data allocated to the validation set can significantly affect the evaluation of the model's performance and its ability to generalize. Here's a breakdown of how increasing or decreasing the validation set size impacts accuracy:

### Increasing the Percentage of the Validation Set

1. **More Reliable Evaluation:**
   - **Increased Precision:** A larger validation set provides a more comprehensive evaluation of the model’s performance across a broader spectrum of data. This generally results in a more accurate estimate of the model's performance metrics (e.g., accuracy, precision, recall).
   - **Reduced Variance:** With more data in the validation set, the evaluation metrics tend to have lower variance, meaning that the performance estimate is less sensitive to random fluctuations in the data.

2. **Fewer Training Examples:**
   - **Potential Overfitting:** As the validation set size increases, the number of examples available for training decreases. This can lead to a less well-trained model if the training set becomes too small. The model might not learn enough from the reduced training data, potentially leading to poorer performance.
   - **Reduced Model Quality:** Smaller training data might result in less effective learning, potentially reducing the overall accuracy of the model.

### Reducing the Percentage of the Validation Set

1. **Less Reliable Evaluation:**
   - **Increased Variance:** A smaller validation set may not capture the full diversity of the data. This can lead to high variance in performance metrics, making the evaluation less stable and less reliable.
   - **Less Accurate Estimates:** With fewer examples, the performance metrics may be more susceptible to noise or outliers in the validation set, potentially giving an inaccurate picture of the model's true performance.

2. **More Training Examples:**
   - **Better Model Training:** With a larger portion of the data allocated to training, the model has more examples to learn from, which can improve its performance and generalization capabilities.
   - **Improved Model Quality:** More training data generally means that the model can better capture the underlying patterns in the data, potentially increasing its accuracy and effectiveness.

### Trade-offs and Best Practices

1. **Balanced Approach:** The key is to strike a balance between having enough data for validation to get a reliable estimate of performance and having sufficient data for training to ensure the model is well-trained. Common practices include using techniques like cross-validation, where multiple validation sets are used to get a more robust estimate of model performance.

2. **Cross-Validation:** In cross-validation, the dataset is divided into multiple folds, and each fold is used as a validation set while the remaining folds are used for training. This helps to mitigate the issue of having too small a validation set and provides a better estimate of model performance.

3. **Model Complexity and Data Size:** The impact of changing the validation set size also depends on the complexity of the model and the overall size of the dataset. For smaller datasets, having a large validation set can be detrimental to training, while for larger datasets, the impact may be less pronounced.

### Summary

- **Increasing the validation set size** generally leads to more reliable performance estimates but reduces the amount of data available for training, which can hurt the model's accuracy if the training data becomes insufficient.
- **Reducing the validation set size** allows for more data to be used for training, potentially improving model performance, but may lead to less stable and less reliable performance estimates.

Balancing these considerations is crucial for achieving a model that not only performs well on unseen data but also provides trustworthy performance metrics during evaluation.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
The size of the train and validation sets plays a crucial role in how well you can estimate the accuracy of a model on the test set. Here’s how different sizes impact the accuracy estimation:

### Impact of Validation Set Size

1. **Accuracy of Performance Estimates:**
   - **Larger Validation Set:** With a larger validation set, the performance metrics (such as accuracy) you obtain are more representative of the model’s performance on unseen data. This is because a larger validation set provides a more comprehensive evaluation and reduces the influence of random fluctuations or noise in the data. Consequently, the accuracy estimated on the validation set is generally more reliable and closer to what you might expect on the test set.
   - **Smaller Validation Set:** A smaller validation set may not capture the full variability of the data. As a result, performance estimates can be less stable and more affected by random noise or outliers. This can lead to inaccuracies in estimating the model's performance on the test set, as the small validation set might not be fully representative of the broader data distribution.

2. **Variance in Performance Estimates:**
   - **Larger Validation Set:** Reduces the variance in performance estimates. A larger sample size provides a better approximation of the true performance of the model, leading to more stable and consistent accuracy metrics.
   - **Smaller Validation Set:** Increases the variance in performance estimates. Performance metrics can fluctuate more due to the smaller sample size, which may not fully represent the diversity of the data.

### Impact of Training Set Size

1. **Model Learning and Generalization:**
   - **Larger Training Set:** Provides more examples for the model to learn from, which generally improves the model’s ability to generalize to new, unseen data. A well-trained model is likely to perform better on the test set. However, the accuracy of the performance estimate from the validation set remains crucial.
   - **Smaller Training Set:** The model may not learn effectively from a smaller amount of data, potentially leading to overfitting or underfitting. This can affect the model’s generalization ability and subsequently impact how well the validation accuracy predicts the test accuracy.

2. **Balance Between Training and Validation:**
   - **Balanced Sets:** Ideally, you want a balance where both the training and validation sets are sufficiently large to ensure the model is well-trained and the performance estimates are reliable. This balance allows for accurate performance evaluation and better generalization to the test set.
   - **Imbalance:** If the training set is very large while the validation set is very small (or vice versa), the performance metrics might not accurately reflect the model’s true performance. An extremely small validation set might not capture the model's performance accurately, while a very small training set might lead to a poorly trained model.

### Summary

- **A larger validation set** provides a more accurate and stable estimate of how well the model will perform on the test set because it reduces the variance in performance metrics and captures a more representative sample of the data.
- **A smaller validation set** can lead to less reliable performance estimates due to increased variance and potential lack of representativeness.
- **A larger training set** generally improves model performance and helps in achieving better generalization, which can also help in obtaining more reliable validation metrics, provided the validation set is also adequately sized.

In practice, using techniques like cross-validation can help mitigate some of these issues by using multiple validation sets and averaging performance estimates. This approach helps in achieving a more robust and reliable evaluation of the model's performance.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?
Choosing the right percentage to reserve for the validation set involves balancing several factors, including the size of the dataset, the complexity of the model, and the need for reliable performance estimates. Here are some general guidelines and common practices to help you decide on a suitable percentage for the validation set:

### General Guidelines

1. **Size of the Dataset:**
   - **Large Datasets:** For large datasets (e.g., thousands or millions of examples), reserving 10% to 20% of the data for validation is typically sufficient. The large size of the dataset ensures that even a smaller percentage of data will provide a reliable performance estimate.
   - **Medium-Sized Datasets:** For datasets with a moderate number of examples (e.g., hundreds to a few thousand), reserving around 20% to 30% of the data for validation is a common practice. This balance provides enough data for validation while still retaining a substantial amount for training.
   - **Small Datasets:** For smaller datasets (e.g., fewer than 1000 examples), a larger percentage may be necessary to get a reliable estimate of performance. In such cases, you might reserve 30% to 40% of the data for validation, but also consider using techniques like cross-validation to maximize the use of limited data.

2. **Model Complexity:**
   - **Simple Models:** For simpler models, a smaller validation set may be sufficient because the model's performance may not be as sensitive to the size of the validation set.
   - **Complex Models:** For more complex models, a larger validation set can help provide a more accurate estimate of performance due to the increased sensitivity of these models to data variations.

3. **Purpose of Validation:**
   - **Hyperparameter Tuning:** If the primary use of the validation set is for hyperparameter tuning, ensure it's large enough to give a good estimate of the model's performance with different hyperparameter settings. Typically, 20% to 30% is a good range.
   - **Performance Evaluation:** If the validation set is used primarily for performance evaluation before testing, aim for a size that ensures reliability without sacrificing too much training data. Again, 20% to 30% is common in practice.

### Common Practices

1. **80/20 Split:**
   - Reserve 80% of the data for training and 20% for validation. This is a widely used split that works well for many scenarios, especially when the dataset is large enough.

2. **70/30 Split:**
   - Reserve 70% of the data for training and 30% for validation. This approach is often used when you want to ensure a more robust validation set, particularly for medium-sized datasets.

3. **Cross-Validation:**
   - For small to medium-sized datasets, cross-validation (e.g., k-fold cross-validation) can be very effective. This method involves splitting the data into multiple folds, using each fold as a validation set while training on the remaining folds. This approach provides a more comprehensive evaluation of model performance and maximizes the use of available data.

4. **Holdout and Cross-Validation Combination:**
   - In some cases, you might use a holdout validation set (e.g., 20% of the data) for initial evaluation and then apply cross-validation on a subset of the remaining data for more detailed tuning and validation.

### Summary

- For **large datasets**, reserving **10% to 20%** of the data for validation is typically effective.
- For **medium-sized datasets**, a **20% to 30%** validation split is commonly used.
- For **small datasets**, consider reserving **30% to 40%** of the data for validation, or use cross-validation to make the most of the limited data.

Ultimately, the exact percentage will depend on your specific use case and dataset size. The key is to ensure that the validation set is large enough to provide reliable performance estimates while leaving sufficient data for training to build an effective model.

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.